In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import pandas as pd
import numpy as np
import math
import matplotlib.pylab as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (16, 9)
plt.style.use('fast')

from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten
from sklearn.preprocessing import MinMaxScaler

In [ ]:
df = pd.read_csv('DATACOPPM.csv',  parse_dates=[0], header=None,index_col=0, squeeze=True,names=['fecha','unidades'])
df.head()

In [ ]:
df.describe()

In [ ]:
print(df.index.min())
print(df.index.max())

In [ ]:
print(len(df['2020-08']))
print(len(df['2020-09']))
print(len(df['2020-10']))
print(len(df['2020-11']))
print(len(df['2020-12']))

In [ ]:
meses =df.resample('M').mean()
meses

In [ ]:
plt.plot(meses['2020'].values)

In [ ]:
iqc = df['2020-08-01':'2020-08-20']
plt.plot(iqc.values)
iqc1 = df['2020-09-01':'2020-09-20']
plt.plot(iqc1.values)
iqc2 = df['2020-10-01':'2020-10-20']
plt.plot(iqc2.values)
iqc3 = df['2020-11-01':'2020-11-20']
plt.plot(iqc3.values)
iqc4 = df['2020-12-01':'2020-12-20']
plt.plot(iqc4.values)

In [ ]:
iqc = df['2020-08-01':'2020-08-20']
plt.plot(iqc.values)
iqc1 = df['2020-09-01':'2020-09-20']
plt.plot(iqc1.values)

In [ ]:
iqc = df['2020-08-01':'2020-08-10']
plt.plot(iqc.values)
iqc1 = df['2020-09-01':'2020-09-10']
plt.plot(iqc1.values)
iqc2 = df['2020-10-01':'2020-10-10']
plt.plot(iqc2.values)
iqc3 = df['2020-11-01':'2020-11-10']
plt.plot(iqc3.values)
iqc4 = df['2020-12-01':'2020-12-10']
plt.plot(iqc4.values)

In [ ]:
PASOS=7

# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg
 
# load dataset
values = df.values
# ensure all data is float
values = values.astype('float32')
# normalize features
scaler = MinMaxScaler(feature_range=(-1, 1))
values=values.reshape(-1, 1) # esto lo hacemos porque tenemos 1 sola dimension
scaled = scaler.fit_transform(values)
# frame as supervised learning
reframed = series_to_supervised(scaled, PASOS, 1)
reframed.head()

In [ ]:
# split into train and test sets
values = reframed.values
n_train_days = 153-(30+PASOS)
train = values[:n_train_days, :]
test = values[n_train_days:, :]
# split into input and outputs
x_train, y_train = train[:, :-1], train[:, -1]
x_val, y_val = test[:, :-1], test[:, -1]
# reshape input to be 3D [samples, timesteps, features]
x_train = x_train.reshape((x_train.shape[0], 1, x_train.shape[1]))
x_val = x_val.reshape((x_val.shape[0], 1, x_val.shape[1]))
print(x_train.shape, y_train.shape, x_val.shape, y_val.shape)

In [ ]:
def crear_modeloFF():
    model = Sequential() 
    model.add(Dense(PASOS, input_shape=(1,PASOS),activation='tanh'))
    model.add(Flatten())
    model.add(Dense(1, activation='tanh'))
    model.compile(loss='mean_absolute_error',optimizer='Adam',metrics=["mse"])
    model.summary()
    return model

In [ ]:
EPOCHS=200

model = crear_modeloFF()

history=model.fit(x_train,y_train,epochs=EPOCHS,validation_data=(x_val,y_val),batch_size=PASOS)

In [ ]:
# Estimacion del rendimiento del modelo
trainScore = model.evaluate(x_train, y_train, verbose=0)
print('Resultado del entrenamiento: %.2f MSE (%.2f RMSE)' % (trainScore[1], math.sqrt(trainScore[1])))
testScore = model.evaluate(x_val, y_val, verbose=0)
print('Resultado del test: %.2f MSE (%.2f RMSE)' % (testScore[1], math.sqrt(testScore[1])))


In [ ]:
results=model.predict(x_val)
print( len(results) )
plt.scatter(range(len(y_val)),y_val,c='g')
plt.scatter(range(len(results)),results,c='r')
plt.title('validate')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.title('loss')
plt.plot(history.history['val_loss'])
plt.title('validate loss')
plt.show()

In [ ]:
plt.title('Accuracy')
plt.plot(history.history['mse'])
plt.show()

In [ ]:
compara = pd.DataFrame(np.array([y_val, [x[0] for x in results]])).transpose()
compara.columns = ['real', 'prediccion']

inverted = scaler.inverse_transform(compara.values)

compara2 = pd.DataFrame(inverted)
compara2.columns = ['real', 'prediccion']
compara2['diferencia'] = compara2['real'] - compara2['prediccion']
compara2.head()

In [ ]:
compara2.describe()

In [ ]:
compara2['real'].plot()
compara2['prediccion'].plot()

In [ ]:
ultimosDias = df['2020-11-16':'2020-11-30']
ultimosDias

In [ ]:
values = ultimosDias.values
values = values.astype('float32')
# normalize features
values=values.reshape(-1, 1) # esto lo hacemos porque tenemos 1 sola dimension
scaled = scaler.fit_transform(values)
reframed = series_to_supervised(scaled, PASOS, 1)
reframed.drop(reframed.columns[[7]], axis=1, inplace=True)
reframed.head(7)

In [ ]:
values = reframed.values
x_test = values[6:, :]
x_test = x_test.reshape((x_test.shape[0], 1, x_test.shape[1]))
print(x_test.shape)
x_test

In [ ]:
def agregarNuevoValor(x_test,nuevoValor):
    for i in range(x_test.shape[2]-1):
        x_test[0][0][i] = x_test[0][0][i+1]
    x_test[0][0][x_test.shape[2]-1]=nuevoValor
    return x_test

In [ ]:
results=[]
for i in range(7):
    parcial=model.predict(x_test)
    results.append(parcial[0])
    print(x_test)
    x_test=agregarNuevoValor(x_test,parcial[0])

In [ ]:
adimen = [x for x in results]    
print(adimen)
inverted = scaler.inverse_transform(adimen)
inverted

In [ ]:
prediccion1SemanaDiciembre = pd.DataFrame(inverted)
prediccion1SemanaDiciembre.columns = ['pronostico']
prediccion1SemanaDiciembre.plot()
prediccion1SemanaDiciembre.to_csv('pronostico.csv')

In [ ]:

prediccion1SemanaDiciembre

In [ ]:
i=0
for fila in prediccion1SemanaDiciembre.pronostico:
    i=i+1
    ultimosDias.loc['2018-12-0' + str(i) + ' 00:00:00'] = fila
    print(fila)
ultimosDias.tail(14)

In [ ]:

from pandas import read_csv
from matplotlib import pyplot
series = read_csv('DATACOPPM.csv', header=0, index_col=0)
series.plot()
pyplot.show()


In [ ]:
from pandas import read_csv
from matplotlib import pyplot
from statsmodels.graphics.tsaplots import plot_acf
series = read_csv('DATACOPPM.csv', header=0, index_col=0)
plot_acf(series)
pyplot.show()

In [ ]:

from pandas import read_csv
from matplotlib import pyplot
from statsmodels.graphics.tsaplots import plot_pacf
series = read_csv('DATACOPPM.csv', header=0, index_col=0)
plot_pacf(series, lags=50)
pyplot.show()